<a href="https://colab.research.google.com/github/devraval/GOT_S08_sentiments/blob/main/NLP_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Jul 20 00:24:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    32W /  70W |   4478MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import os
import datetime
import tensorflow_hub as hub
import numpy as np
    
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Kaggle/imdb_reviews/GOT_S08_imdb_reviews.csv',index_col=0)

In [ ]:
df.tail(30)

,Unnamed: 0.1,rating,content
756144,756145,1,Absolute nonsense franklf May What is this SXX...
756145,756146,1,i can't believe this end medogb May The end mo...
756146,756147,8,I can't say anything 😰🔥 abdalla- May The episo...
756147,756148,4,The ending that got people to read more books....
756149,756150,1,I have to give it fare review ikfooii May Well...
756150,756151,10,What do you people expect?! scasey- May What ...
756151,756152,2,I never been disappointed this much saadzafar-...
756152,756153,7,The Greatest Show to Ever Exist miller_christi...
756153,756154,1,what was the reason for everything? ngonzaleza...
756154,756155,3,Disappointing torirockshard May Part of me wis...


In [ ]:
pd.set_option('max_colwidth',-1)
df.tail(30)

In [ ]:
df['content'] = df['content'].apply((lambda x: re.sub('[^a-z0-9\s?!]','',x.lower())))
df['rating']=df['rating'].astype(int,copy=False)
df.drop('Unnamed: 0.1',1,inplace=True)
max_fatures = 2500
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['content'].values)
X1 = tokenizer.texts_to_sequences(df['content'].values)
X = pad_sequences(X1)

In [ ]:
vocab=tokenizer.get_config()
vocab

In [ ]:
df['rating'].value_counts()

### Target encoding
0 : Absolutely negative sentiment with 1,2,3,4 stars

1 : Fairly negative sentiment with 5, 6 stars

2 : Mixed, fairly positive sentiment with 7,8 stars

3 : Absolutely Positive sentiment with 9,10 stars

In [ ]:
df['rating'].replace([1,2,3],0,inplace=True)
df['rating'].replace([4,5],1,inplace=True)
df['rating'].replace([6,7],2,inplace=True)
df['rating'].replace([8,9],3,inplace=True)
df['rating'].replace([10],4,inplace=True)

In [ ]:
from sklearn.utils import class_weight
class_weights = list(class_weight.compute_class_weight('balanced',
                                             np.unique(df['rating']),
                                             df['rating']))

In [ ]:
class_weights

In [ ]:
X_train, X_valid_test = train_test_split(df, test_size=0.3, shuffle=True,random_state=11)

In [ ]:
X_valid,X_test=train_test_split(X_valid_test,test_size=0.5,shuffle=True,random_state=11)

In [ ]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train['content'].values, X_train['rating'].values))
dataset_valid = tf.data.Dataset.from_tensor_slices((X_valid['content'].values, X_valid['rating'].values))
dataset_test = tf.data.Dataset.from_tensor_slices((X_test['content'].values, X_test['rating'].values))

In [ ]:
for text, target in dataset_train.take(5):
  print ('content: {} \n rating: {}'.format(text, target))

In [ ]:
for text, target in dataset_test.take(5):
  print ('content: {} \n rating: {}'.format(text, target))

content: b' the best got episode shtain april a great course of action great acting great effects and nontheless a great score                     out of  found this helpful                                                    was this review helpful?  sign in to vote                        permalink' 
 rating: 10
content: b' for those who rated  star jokmokri may please rate professionally and dont rushif you dont like it just give it a fair score                     out of  found this helpful                                                    was this review helpful?  sign in to vote                        permalink' 
 rating: 10
content: b' one of the best episodes of the entire got dimitrie may warning spoilers                     out of  found this helpful                                                    was this review helpful?  sign in to vote                        permalinki dont understand all these very bad reviewsyes previous episodes of these season were not as we expected

In [ ]:
def fetch(text, labels):
  return text, tf.one_hot(labels,10)

In [ ]:
train_data_f = dataset_train.map(fetch)
valid_data_f = dataset_valid.map(fetch)
test_data_f = dataset_test.map(fetch)

In [ ]:
train_data, train_labels = next(iter(train_data_f.batch(5)))
train_data, train_labels

(<tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b'poor written tgbao may the saddest death of this episode was this series extremely disappointed                     out of  found this helpful                                                    was this review helpful?  sign in to vote                        permalink',
        b'doesnt make any sense whitegio may warning spoilers                     out of  found this helpful                                                    was this review helpful?  sign in to vote                        permalinkthe characters behave completely out of character it is just not believable and the way theyve killed the dragon?!? such bs the plot doesnt make any sense and the story doesnt flow they seem to teleport everywhere when in the first seasons it would take  episodes to move from one castle to another one they dont convey any passage of time between scenes they have butchered so many characters who had evolved so much during the series now

In [ ]:
test_data, test_labels = next(iter(test_data_f.batch(5)))
test_data, test_labels

(<tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b' the best got episode shtain april a great course of action great acting great effects and nontheless a great score                     out of  found this helpful                                                    was this review helpful?  sign in to vote                        permalink',
        b' for those who rated  star jokmokri may please rate professionally and dont rushif you dont like it just give it a fair score                     out of  found this helpful                                                    was this review helpful?  sign in to vote                        permalink',
        b' one of the best episodes of the entire got dimitrie may warning spoilers                     out of  found this helpful                                                    was this review helpful?  sign in to vote                        permalinki dont understand all these very bad reviewsyes previous episodes of these season were

In [ ]:
import tensorflow_hub as hub
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(embedding, output_shape=[128], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_data[:1])

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
for units in [128, 64 , 32]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))
  model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
train_data_f=train_data_f.shuffle(70000).batch(512)
test_data_f=test_data_f.batch(512)

In [ ]:
history = model.fit(train_data_f,
                    epochs=4,
                    validation_data=valid_data_f.batch(512),
                    verbose=1)

In [ ]:
len(list(dataset_test))

73570

In [ ]:
results = model.evaluate(dataset_test.map(fetch).batch(11491), verbose=2)

print(results)

7/7 - 2s - loss: 0.0017 - accuracy: 0.9996
[0.0016712306533008814, 0.9996194243431091]


In [ ]:
test_data1, test_labels1 = next(iter(dataset_test.map(fetch).batch(45963)))

In [ ]:
y_pred=model.predict(test_data1)

from sklearn.metrics import classification_report

print(classification_report(test_labels1.numpy().argmax(axis=1), y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23412
           1       1.00      1.00      1.00      4427
           2       1.00      1.00      1.00      3904
           3       1.00      1.00      1.00     14220

    accuracy                           1.00     45963
   macro avg       1.00      1.00      1.00     45963
weighted avg       1.00      1.00      1.00     45963

